In [ ]:
import os
import sys
from time import sleep

import numpy as np

import GCode
import GRBL


# Laser Material Calibration

Loop over laser power settings and feed rate to determine how a given material looks.

# Code:

In [2]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
sleep(1)
cnc.laser_mode = 1


ok


In [3]:
print(cnc.laser_mode)


1.0


In [4]:
cnc.status


'<Idle|MPos:0.000,0.000,0.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>'

In [5]:
cnc.kill_alarm()


In [6]:
def laser_aim(self):
    self.cmd("G1F1")
    self.cmd("M3S1")
    input("Aim laser and press enter to continue...")
    self.cmd("M3S0")
    self.cmd("M5")


In [7]:
laser_aim(cnc)


Aim laser and press enter to continue...


In [8]:
def init():
    program = GCode.GCode()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0)
    return program


In [9]:
def line(X=0, Y0=0, Yf=20, feed=100, power=255, dynamic_power=True):
    prog = GCode.GCode()
    prog.G0(X=X, Y=Y0)
    if dynamic_power:
        prog.M4(S=power)
    else:
        prog.M3(S=power)
    prog.G1(X=X, Y=Yf, F=feed)
    prog.M5()
    return prog


In [26]:
dX = 0.1
Xf = 2
dXs = np.arange(dX, Xf + dX, dX)
dXs


array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ])

In [ ]:
cnc.reset()


In [ ]:
X0 = 0
line_len = 15
X = X0
Yf = 0

for feed in [50, 100, 200, 300]:
    Y0 = Yf
    Yf = Y0 + line_len
    for power in [50, 100, 150, 200, 255]:
        for X in np.cumsum(dXs) + X0:
            L = line(X=X, Y0=Y0, Yf=Yf, feed=feed, power=power)
            cnc.run(L)
        X0 = X + 5


^C


In [48]:
L0 = line(X=0, Y0=Y0, Yf=Yf, feed=feed, power=255)
L1 = line(X=2, Y0=Y0, Yf=Yf, feed=feed, power=255)

cnc.run(L0)
cnc.run(L1)

for X, power in zip(Xs, powers):
    L = line(X=X, Y0=Y0, Yf=Yf, feed=feed, power=power)
    cnc.run(L)


In [49]:
cnc.cmd("G0X0Y0")


['ok', 'ok']